# Improving Quantum Genetic Optimization through Granular Computing
### by Giovanni Acampora and Autilia Vitiello
### Submitted to Granular Computing

#### Abstract
Quantum computers promise to revolutionize the world of computing thanks to some features of quantum mechanics that can enable massive parallelism in computation. 
This benefit may be particularly relevant in the design of evolutionary algorithms, where the quantum paradigm could support the exploration of multiple regions of the search space in a concurrent way. Although some efforts in this research field are ongoing, the potential of quantum computing is not yet fully expressed due to the limited number of qubits of current quantum processors. This limitation is even more acute when one wants to deal with continuous optimization problems, where the search space is potentially infinite. The goal of this paper is to address this limitation by introducing a hybrid and granular approach to quantum algorithm design, specifically designed for genetic optimization. This approach is defined as hybrid because it uses a digital computer to evaluate fitness functions, and a quantum processor to evolve the genetic population; moreover, it uses granular computing to hierarchically reduce the size of the search space of a problem, so that good near-optimal solutions can be identified even on small quantum computers. This notebook allows researchers in quantum computing and artificial intelligence to test our proposal on a set of well-know benchmark functions and to compare performance against the quantum state-of-the-art approach, named HQGA.

#### Importing Modules
The first step to run HGQGA is to import the required Python modules.

In [ ]:
from qiskit import IBMQ
from qiskit import Aer
from math import pi
import matplotlib.pyplot as plt

import Problems as p
import hqga_algorithm
import hqga_iterative
import hqga_utils_extensions
import hqga_utils
import utils
from plotFunctions2D import display

#### Setting the problem
The second step is to define the problem to be solved by HGQGA. You can select among ten problems implemented as different Python functions and referred as to *ProblemF1*, *ProblemF2*, *ProblemF3*, *ProblemF4*, *ProblemF5*, *ProblemF6*, *ProblemF7*, *ProblemF8*, *ProblemF9* and *ProblemF10*. 

The variable *b* is used in the discretization procedure.

In [ ]:
b = 5

# you can change the string ProblemF1 with another one among: 
# ProblemF2, ProblemF3, ProblemF4, ProblemF5, ProblemF6, ProblemF7, ProblemF8, ProblemF9, Problem F10

problem=p.ProblemF1(num_bit_code=b)
display(problem)

#### Defining the quantum backend
The third step is to define the quantum backed to be used to run HGQGA. Our experiments exploit the IBM quantum processor named *Guadalupe* whose access has been provided by IBM in the context of the *IBM Quantum Researchers Program Access Award* (AgreementNumber: W2177387). However, it is possible for all researchers that want test our approach to simulate HGQGA by using a quantum simulator.

In [ ]:
backend = Aer.get_backend('aer_simulator')
device_features=utils.device(backend)

#### Setting HGQGA hyperparameters
The fourth step is to set the hyper-parameters of HGQGA. In this example, the number of chromosomes *m* is set to 3, the maximum number of iterations *max_gen* is set to 5, the maximum number of levels *k* is set to 4, the $\delta$ value used during the initialization procedure is set to $\frac{\pi}{8}$, the $\mu$ value representing
the probability to apply the mutation is set to 0.3 and the $\rho$ value used when the reinforcement elitism is applied is set to $\frac{\pi}{16}$. The selected elitism is the quantum elitism. You can change the elitism strategy by setting the variable *elitism_mod* to ELITISM_D for the deterministic elitism or to ELITISM_R for the quantum elitism with reinforcement.

In [ ]:
#hyper-parameters in the case of the quantum elitism
m = 3
max_gen = 5
k = 4
delta = pi/16
mu = 0.3
rho = pi/16
elitism_mod = hqga_utils.ELITISM_Q


if elitism_mod == hqga_utils.ELITISM_Q:
    params = hqga_utils_extensions.ExtensionsParameters(k, m, max_gen, delta, mu, elitism=hqga_utils.ELITISM_Q)
elif elitism_mod == hqga_utils.ELITISM_D:
    params = hqga_utils_extensions.ExtensionsParameters(k, m, max_gen, delta, mu, elitism=hqga_utils.ELITISM_D)
elif elitism_mod == hqga_utils.ELITISM_R:
    params = hqga_utils_extensions.ExtensionsReinforcementParameters(k, m, max_gen, delta, rho, mu)
else:
    print("Please, select one elitism procedure among ELITISM_Q, ELITISM_D and ELITISM_R")

#### Setting visualization parameters
This setting allows users to obtain the visualization of the circuits (*draw_circuit=True*), the visualization of the classical chromosomes and the relative fitness values (*verbose = True*) and, finally, the visualization of a progress bar to monitor the number of iterations (*progressBar = True*).

In [ ]:
#presentation hyper-parameters
params.progressBar = True
params.verbose = True
params.draw_circuit = True

#### Setting up HGQGA quantum circuit
The fifth step is to create an empty circuit with the required number of qubits.

In [ ]:
# build circuit
circuit = hqga_utils.setupCircuit(params.pop_size, problem.dim * problem.num_bit_code)
#print(circuit)

#### Executing HGQGA
The last step is to run H2QGA. At the end of the execution, this code displays the best solution, the optimal solution to the problem, the fitness function value, the optimal fitness function value in order to assess the quality of the generated solution. Moreover, in order to compare the performance of HGQGA with HQGA, also the solution and its fitness function value for HQGA are reported.

In [ ]:
# start test
gBest, list_bests = hqga_iterative.runQGA(device_features, circuit, params, problem)
opt_sol, opt_fitness = problem.getOptimalSol(k)
print("Best solution - HGQGA",gBest.chr)
print("Optimal solution",opt_sol)
print("Best solution - HQGA",str(problem.convert(list_bests[0].chr)))
print("Fitness of the best solution - HGQGA",gBest.fitness)
print("Optimal fitness", opt_fitness)
print("Fitness of the best solution - HQGA",list_bests[0].fitness)

#### Systematic comparison
To perform a systematic comparison between H2QGA and HQGA, we compare the fitness values on several runs. For this comparison, the number of runs is set to 25.

In [ ]:
params.draw_circuit = False
params.verbose = False

num_runs=5

h2qga_fitnesses=[]
hqga_fitnesses=[]

# start comparison
for i in range(num_runs):
    hqga_utils.resetCircuit(circuit)
    gBest, list_bests = hqga_iterative.runQGA(device_features, circuit, params, problem)
    opt_sol, opt_fitness = problem.getOptimalSol(k)
    
    print("Best solution - HGQGA",gBest.chr)
    print("Optimal solution",opt_sol)
    print("Best solution - HQGA",str(problem.convert(list_bests[0].chr)))
    print("Fitness of the best solution - HGQGA",gBest.fitness)
    print("Optimal fitness", opt_fitness)
    print("Fitness of the best solution - HQGA",list_bests[0].fitness)
    h2qga_fitnesses.append(gBest.fitness)
    hqga_fitnesses.append(list_bests[0].fitness)

#display boxplots
data =[h2qga_fitnesses]+[hqga_fitnesses]
labels=["HGQGA"] + ["HQGA"]
meanlineprops = dict(linestyle='-', linewidth=1, color='blue')
meanpointsprops = dict(marker='.', markersize=5, markeredgecolor='firebrick', markerfacecolor='firebrick')
plt.boxplot(data, sym="b+", labels=labels, vert=True, showfliers=True, showmeans=True, patch_artist=True, boxprops=dict(facecolor="lightgray", linewidth="1"),medianprops=dict(color="firebrick", linewidth="1"), meanprops=meanpointsprops)
plt.tick_params(axis='y', labelsize="8")
plt.tick_params(axis='x', rotation=0,labelsize="8")
plt.show()

<h1><center>QUASAR Laboratory. University of Naples Federico II.</center></h1>